# Reading data using `readlines`

Look at an example file from [NWIS site inventory](https://nwis.waterdata.usgs.gov/nwis/inventory\n') that contains a list of all USGS sites in the Madison area. 

In [ ]:
input_file = 'data/site_information.txt'

In [ ]:
with open(input_file, 'r') as f:
    lines = f.readlines()

In [ ]:
lines

`f.readlines()` reads the file line-by-line, returning a list of strings (for every line in the file)

In [ ]:
lines[24]

Split line on whitespace

In [ ]:
lines[24].split()

Split line on tabs

In [ ]:
lines[24].split('\t') # '\t' is the tabs character

### Get list of the surface water sites only

In [ ]:
sw_site_lines = []

for line in lines:
    if line.startswith('#'): # skip the header lines
        continue
    else:
        parts = line.split("\t")
        if parts[1].startswith('054'): # use the site_no to differentiate sw from gw -- starts with "054"
            sw_site_lines.append(parts)

In [ ]:
sw_site_lines

In [ ]:
sw_site_lines[0]

In [ ]:
sw_site_lines[0][1]

### Activity: Can you make a list of all surface water site numbers?

## Let's write the surface water lines out to a new file using tab separators

We can add the tabs back between the items using `join()`

In [ ]:
sw_site_lines[0]

In [ ]:
'\t'.join(sw_site_lines[0])

make a new directory called output

In [ ]:
import os

# make a new folder for output files
if not os.path.exists('output'):  # if it doesn't already exist
    os.mkdir('output')

In [ ]:
with open('output/sw_site_info_tabs.txt', 'w') as f:
    for line in sw_site_lines:
        f.write('\t'.join(line))

### How about a comma separated .csv file?

In [ ]:
with open('output/sw_site_info_commas.csv', 'w') as f:
    for line in sw_site_lines:
        f.write(','.join(line)) # just join on ',' instead of tab

### Slightly annoying - Excel splits on the comma in the station name! 
Add an extra step to remove commas before writing to csv

In [ ]:
with open('output/sw_site_info_commas_fixed.csv', 'w') as f:
    for line in sw_site_lines:

        updated_line = []
        for item in line:
            updated_line.append(item.replace(',', ' ')) # replace commas with a space
            
        f.write(','.join(updated_line))

### finally, add a header line to the file

In [ ]:
with open('output/sw_site_info_commas_header.csv', 'w') as f:
    
    f.write('agency_cd,site_no,site_name\n') # this new header line needs the line ending character ('\n') at the end
    
    for line in sw_site_lines:
        
        updated_line = []
        for item in line:
            updated_line.append(item.replace(',', ' '))
            
        f.write(','.join(updated_line))

# Reading data using `Pandas`

In [ ]:
import pandas as pd

### Pandas' `read_csv()` is the primay way to read data into a Pandas DataFrame

`read_csv()` is geared to read comma-separated values (csv) files

In [ ]:
df = pd.read_csv('output/sw_site_info_commas_header.csv')
df

but `read_csv()` can also easily be used to read tabular data from a wide range of formats by specifying additional information to parse the file. 

Here, we read in the original text file using pandas and specify custon column names

In [ ]:
df = pd.read_csv('data/site_information.txt', 
                 delimiter='\t', 
                 skiprows=24, # skip commented out header lines
                 names=['agency_cd', 'site_number', 'site_name'])
df

## Reading from Excel worksheets using `Pandas`

In [ ]:
input_data = 'data/daily_data.xlsx'

This excel file contains mean daily flows data from two gages in separate tabs:
- [USGS 040851385 FOX RIVER AT OIL TANK DEPOT AT GREEN BAY, WI](https://waterdata.usgs.gov/wi/nwis/dv/?site_no=040851385&referred_module=sw)
- [USGS 05407000 WISCONSIN RIVER AT MUSCODA, WI](https://waterdata.usgs.gov/nwis/uv?site_no=05407000&legacy=1)

We can look at the names of these tabs uing Pandas:

In [ ]:
xl = pd.ExcelFile(input_data)
xl.sheet_names

Next, we can use Pandas to read data from a specific sheet using `read_excel()`

In [ ]:
fox_df = pd.read_excel(input_data, sheet_name='040851385')
fox_df

You can lose the leading zero from the site_no during this step (Excel does it too) when Pandas interprets the site_no as an integer

In [ ]:
fox_df.dtypes

if you want, you can add the leading zero back to the `site_no` column and convert it to string (so that it doesn't happen again)

In [ ]:
fox_df['site_no'] = [f'{i:09}' for i in fox_df.site_no] # list comprehension with f-string formatting
fox_df

## Finally, we can save the data from this tab to it's own .csv file using `to_csv()`

In [ ]:
fox_df.to_csv('output/040851385_mean_daily_flow.csv', index=False) # if index=True then the 0-365 indicies (left) are saved in the file too